## Analyze STDP update dynamics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from spikey.synapse import *
from spikey.neuron import *

SYNAPSE = RLSTDPET
NEURON = Neuron

In [ ]:
## Seeing if output rate = inputrate * w given 0 decay
config = {
    'n_neurons': 1,
    'n_inputs': 1,
    'potential_decay': 0, ## TODO VARY THIS
    'fire_threshold': 1,
    'magnitude': 1,
    'neuron_pct_inhibitory': 0,
    'prob_rand_fire': 0,
    'refractory_period': 0,
    'resting_mv': 0,
    'spike_delay': 0,
}

neurons = NEURON(**config)

LENGTH = 100
INHIBITORIES = np.array([1, 1])

RATES = [0, 1, 10, 20, 30, 40, 50]  # hz, number of fires / interval -- evenly spaced

for inrate in RATES:
    X, Y = [], []

    for weight in np.arange(0, 1.1, .1):
        neurons.reset()

        #neuron += [inrate * w]

        a = np.zeros(LENGTH)
        if inrate:
            a[::LENGTH // inrate] = 1.

        spike_train = []
        for value in a:
            neurons += value * weight
            spike_train.append((neurons >= config['fire_threshold'])[0])
            neurons.update()

        outrate = np.mean(spike_train)

        X.append(weight)
        Y.append(outrate)

    plt.plot(X, Y, label=f"pre={np.mean(a)}")

plt.title(f"pre - post rate dW")
plt.xlabel('weights')
plt.ylabel('output rate')
plt.legend()

In [ ]:
## Presynaptic rate vs w vs trace
config = {
    'n_neurons': 2,
    'n_inputs': 0,
    'stdp_window': 150,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': None,
    'magnitude': 1,
    'neuron_pct_inhibitory': 0,
    'potential_decay': 0,
    'prob_rand_fire': 0,
    'refractory_period': 0,
    'resting_mv': 0,
    'spike_delay': 0,
    'fire_threshold': 1,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = [1, 25, 50, 75, 100, 125, 150]  # hz, number of fires / interval -- evenly spaced

for weight in np.arange(0, 1.1, .1):
    X, Y = [], []

    for rate1 in RATES:
        synapses.reset()

        a = np.zeros(LENGTH)
        a[::LENGTH // rate1] = 1.

        neurons = NEURON(**config)
        neurons.reset()
        b = []
        for i, value in enumerate(a):
            neurons += value * weight
            b.append((neurons >= config['fire_threshold'])[0])
            neurons.update()

            full_spike_log = np.array([a[:i+1], np.array(b)]).T

            synapses._apply_stdp(full_spike_log, INHIBITORIES)

        X.append(rate1)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"weight={np.round(weight, 2)}")

plt.title(f"pre - post rate dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
## Seeing if output rate = inputrate[uniform] * w given 0 decay
config = {
    'n_neurons': 1,
    'n_inputs': 1,
    'potential_decay': 0, ## TODO VARY THIS
    'fire_threshold': 1,
    'magnitude': 1,
    'neuron_pct_inhibitory': 0,
    'prob_rand_fire': 0,
    'refractory_period': 0,
    'resting_mv': 0,
    'spike_delay': 0,
}

neurons = NEURON(**config)

LENGTH = 100
INHIBITORIES = np.array([1, 1])

RATES = [0, 1, 10, 20, 30, 40, 50]  # hz, number of fires / interval -- evenly spaced

for inrate in RATES:
    X, Y = [], []

    for weight in np.arange(0, 1.1, .1):
        neurons.reset()

        #neuron += [inrate * w]

        a = np.int_(np.random.uniform(0, 1, size=LENGTH) <= inrate / LENGTH)

        spike_train = []
        for value in a:
            neurons += value * weight
            spike_train.append((neurons >= config['fire_threshold'])[0])
            neurons.update()

        outrate = np.mean(spike_train)

        X.append(weight)
        Y.append(outrate)

    plt.plot(X, Y, label=f"pre={np.mean(a)}")

plt.title(f"pre - post rate dW")
plt.xlabel('weights')
plt.ylabel('output rate')
plt.legend()

In [ ]:
## Presynaptic rate uniform vs w vs trace
config = {
    'n_neurons': 2,
    'n_inputs': 0,
    'stdp_window': 150,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': None,
    'magnitude': 1,
    'neuron_pct_inhibitory': 0,
    'potential_decay': 0,
    'prob_rand_fire': 0,
    'refractory_period': 0,
    'resting_mv': 0,
    'spike_delay': 0,
    'fire_threshold': 1,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = [1, 25, 50, 75, 100, 125, 150]  # hz, number of fires / interval -- evenly spaced

for weight in np.arange(0, 1.1, .1):
    X, Y = [], []

    for rate1 in RATES:
        synapses.reset()

        a = np.int_(np.random.uniform(0, 1, size=LENGTH) <= rate1 / LENGTH)

        neurons = NEURON(**config)
        neurons.reset()
        b = []
        for i, value in enumerate(a):
            neurons += value * weight
            b.append((neurons >= config['fire_threshold'])[0])
            neurons.update()

            full_spike_log = np.array([a[:i+1], np.array(b)]).T

            synapses._apply_stdp(full_spike_log, INHIBITORIES)

        X.append(rate1)
        Y.append(synapses.trace[1, 0])

    #plt.plot(X, Y, label=f"weight={np.round(weight, 2)}")
    plt.plot(np.unique(X), np.poly1d(np.polyfit(X, Y, 1))(np.unique(X)), label=f"weight={np.round(weight, 2)}")

plt.title(f"pre - post rate dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
## Presynaptic rate uniform vs w vs trace + decay, refractory
config = {
    'n_neurons': 2,
    'n_inputs': 0,
    'stdp_window': 150,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': None,
    'magnitude': 1,
    'neuron_pct_inhibitory': 0,
    'potential_decay': .2,
    'prob_rand_fire': 0,
    'refractory_period': 6,
    'resting_mv': 0,
    'spike_delay': 0,
    'fire_threshold': 1,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = [1, 25, 50, 75, 100, 125, 150]  # hz, number of fires / interval -- evenly spaced

for weight in np.arange(0, 1.1, .1):
    X, Y = [], []

    for rate1 in RATES:
        synapses.reset()

        a = np.int_(np.random.uniform(0, 1, size=LENGTH) <= rate1 / LENGTH)

        neurons = NEURON(**config)
        neurons.reset()
        b = []
        for i, value in enumerate(a):
            neurons += value * weight
            b.append((neurons >= config['fire_threshold'])[0])
            neurons.update()

            full_spike_log = np.array([a[:i+1], np.array(b)]).T

            synapses._apply_stdp(full_spike_log, INHIBITORIES)

        X.append(rate1)
        Y.append(synapses.trace[1, 0])

    #plt.plot(X, Y, label=f"weight={np.round(weight, 2)}")
    plt.plot(np.unique(X), np.poly1d(np.polyfit(X, Y, 1))(np.unique(X)), label=f"weight={np.round(weight, 2)}")

plt.title(f"pre - post rate dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()